<a href="https://colab.research.google.com/github/WasikulIslam1/CIS-4400-Project-9/blob/main/LivingwageETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install azure-storage-blob
!pip install pyarrow
import pandas as pd
import numpy as np
import json
import requests
from io import StringIO
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
import datetime
import calendar

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.5/394.5 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.4/193.4 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 2.1 MB/s eta 0:00:00


In [2]:


# Read Azure Blob Storage connection string from config.json
with open('config.json', 'r') as config_file:
    config = json.load(config_file)

# Get connection string from config
connection_string = config['connectionString']

# Create BlobServiceClient
blob_service_client = BlobServiceClient.from_connection_string(connection_string)
container_name = 'livingwage'
blob_name = 'fbclivingwage'
download_file_path = 'fbclivingwage.csv'

blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)

with open(download_file_path, 'wb') as download_file:
    download_file.write(blob_client.download_blob().readall())

print(f"Blob {blob_name} has been downloaded")

Blob fbclivingwage has been downloaded


In [3]:
df = pd.read_csv('fbclivingwage.csv')
Data_df = df.copy()
pd.set_option('display.max_columns', None)

# Set the column names to be the values of the third row
Data_df.columns = Data_df.iloc[2]

# Drop the first three rows
Data_df = Data_df.iloc[3:]





In [7]:

LW_df = Data_df[['state_alpha','county_fips', 'county_name', 'Min', 'Median', 'Max']].copy()


df = LW_df.rename(columns={
    "Min": "minlivingwage",
    "Max":"maxlivingwage",
    "Median": "avglivingwage",
    "county_fips":"county_fips"})

new_order = ["county_fips",
             "minlivingwage",
             "maxlivingwage",
             "avglivingwage"]

livingwages = df[new_order]
file_path = "livingWages.csv"
# Convert DataFrame to CSV
livingwages.to_csv(file_path, index=False)
livingwages

2,county_fips,minlivingwage,maxlivingwage,avglivingwage
3,1001,17.7720955236383,45.9229134095577,26.076836924592
4,1003,18.8079419517347,48.3125390826664,27.7694722630032
5,1005,15.1171447659529,38.1765045656779,21.4595952485424
6,1007,16.6778335257983,43.3086060787738,24.3275029526586
7,1009,16.1355074869611,45.5821325625304,25.4995408849517
...,...,...,...,...
3141,56037,18.1060142636743,48.0164126504615,26.1482516618326
3142,56039,22.5401980805613,67.4130995825591,38.8262227050725
3143,56041,16.7363499805201,45.0327789646043,24.9382882506544
3144,56043,16.6837450624618,45.7940274646545,25.095740373895


In [22]:
# Load Azure Storage connection string and container name from config.json
with open('config.json', 'r') as f:
    config = json.load(f)

connection_string = config['connectionString']


# Path to your local CSV file
container_name = 'livingwage'
file_path = "fbc_livingwage_data_clean.csv"
blob_name = "fbclivingwage_cleaned"

# Create a blob service client
blob_service_client = BlobServiceClient.from_connection_string(connection_string)

# Create a blob client using the container and blob name
blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)

# Upload the file
with open(file_path, "rb") as data:
    blob_client.upload_blob(data)